In [1]:

###############################
##### importing libraries #####
###############################

import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset
torch.backends.cudnn.benchmark=True

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
##### Hyperparameters for federated learning #########
num_clients = 20
num_selected = 6
num_rounds = 150
epochs = 5
batch_size = 32

In [3]:
data = np.load('cifar10_mixed/encryption.npy')
labels = np.load('cifar10_mixed/label.npy')

In [4]:
import numpy as np
from PIL import Image as im

In [5]:
data[0].shape

(32, 32, 3)

In [6]:
image = im.fromarray(data[1], 'RGB')

In [7]:
image.save('sample2.png')

In [8]:
labels[90]

array([0.16742045, 0.        , 0.        , 0.16031678, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)

In [9]:
data.shape

(5000, 32, 32, 3)

In [10]:
dataset = []
add = 0
for i, image in enumerate(data):
    dataset.append((torch.tensor(np.transpose(image)), np.argmax(labels[i])))

In [11]:

#############################################################
##### Creating desired data distribution among clients  #####
#############################################################

# Image augmentation 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Loading CIFAR10 using torchvision.datasets
traindata = np.array(dataset)
# train_fake_data = datasets.CIFAR10('./data', train=True, download=True)

# Dividing the training data into num_clients, with each client having equal number of images
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.shape[0] / num_clients) for _ in range(num_clients)])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

# Normalizing the test images
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Loading the test iamges and thus converting them into a test_loader
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        ), batch_size=batch_size, shuffle=True)

/tmp/ipykernel_2957/3255302352.py:14: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  traindata = np.array(dataset)
/tmp/ipykernel_2957/3255302352.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  traindata = np.array(dataset)


In [12]:
# images, labels = next(iter(train_loader[0]))

In [13]:
dataset = []
add = 0
for i, image in enumerate(data):
    dataset.append((torch.tensor(np.transpose(image)), labels[i]))
dataset = np.asarray(dataset)

/tmp/ipykernel_2957/1474642160.py:5: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  dataset = np.asarray(dataset)
/tmp/ipykernel_2957/1474642160.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.asarray(dataset)


In [14]:
dataset.shape

(5000, 2)

In [15]:
traindata

array([[tensor([[[-0.4124, -0.3718, -0.5027,  ...,  0.2789,  0.2813,  0.2092],
                 [-0.4310,  0.3914,  0.4188,  ...,  0.2574,  0.3305,  0.2537],
                 [ 0.4082,  0.4828, -0.3940,  ...,  0.1587, -0.3160, -0.3205],
                 ...,
                 [ 0.5846,  0.5688, -0.5358,  ..., -0.3715,  0.4111, -0.4737],
                 [-0.5797, -0.5816, -0.5185,  ...,  0.4097,  0.3094, -0.2846],
                 [ 0.5907,  0.6159,  0.4425,  ..., -0.3688,  0.3016, -0.2628]],

                [[-0.3862,  0.3681,  0.5043,  ..., -0.2559, -0.2464,  0.1802],
                 [-0.4079,  0.3867,  0.4276,  ..., -0.2452, -0.3100, -0.2386],
                 [-0.3863,  0.4724,  0.4083,  ..., -0.1491, -0.3035, -0.3180],
                 ...,
                 [ 0.5922, -0.5747,  0.5272,  ...,  0.3122,  0.3473,  0.3944],
                 [ 0.6031, -0.6036, -0.5289,  ...,  0.3972, -0.2938,  0.2614],
                 [-0.6099, -0.6417, -0.4646,  ..., -0.3911,  0.3269, -0.3030]],

    

In [16]:
np.array((data, labels))

/tmp/ipykernel_2957/439118079.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array((data, labels))


ValueError: could not broadcast input array from shape (5000,32,32,3) into shape (5000,)

In [14]:
traindata[0]

array([tensor([[[-0.4124, -0.3718, -0.5027,  ...,  0.2789,  0.2813,  0.2092],
                [-0.4310,  0.3914,  0.4188,  ...,  0.2574,  0.3305,  0.2537],
                [ 0.4082,  0.4828, -0.3940,  ...,  0.1587, -0.3160, -0.3205],
                ...,
                [ 0.5846,  0.5688, -0.5358,  ..., -0.3715,  0.4111, -0.4737],
                [-0.5797, -0.5816, -0.5185,  ...,  0.4097,  0.3094, -0.2846],
                [ 0.5907,  0.6159,  0.4425,  ..., -0.3688,  0.3016, -0.2628]],

               [[-0.3862,  0.3681,  0.5043,  ..., -0.2559, -0.2464,  0.1802],
                [-0.4079,  0.3867,  0.4276,  ..., -0.2452, -0.3100, -0.2386],
                [-0.3863,  0.4724,  0.4083,  ..., -0.1491, -0.3035, -0.3180],
                ...,
                [ 0.5922, -0.5747,  0.5272,  ...,  0.3122,  0.3473,  0.3944],
                [ 0.6031, -0.6036, -0.5289,  ...,  0.3972, -0.2938,  0.2614],
                [-0.6099, -0.6417, -0.4646,  ..., -0.3911,  0.3269, -0.3030]],

               [[ 

In [15]:
traindata[0][1]

6

In [71]:
# images[0]

In [15]:

#################################
##### Neural Network model #####
#################################

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        output = F.log_softmax(out, dim=1)
        return output

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [16]:
def client_update(client_model, optimizer, train_loader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

In [17]:

def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [18]:

def test(global_model, test_loader):
    """This function test the global model on test data and returns test loss and test accuracy """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = global_model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset)

    return test_loss, acc

In [19]:

############################################
#### Initializing models and optimizer  ####
############################################

#### global model ##########
global_model =  VGG('VGG19').cuda()

############## client models ##############
client_models = [ VGG('VGG19').cuda() for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model 

############### optimizers ################
opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

In [20]:
###### List containing info about learning #########
losses_train = []
losses_test = []
acc_train = []
acc_test = []
# Runnining FL

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    losses_train.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)
    
    test_loss, acc = test(global_model, test_loader)
    losses_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))


  0%|          | 0/6 [00:00<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

### 50 Rounds
100 standard deviation gaussian noise accuracy hovered around .163, (all samples)

similar results with half samples at 50 std.

20% application on 20 std accuracy a little better at .2

std 3 80% application accuracy close to .74